In [15]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix

import explore
import modeling

In [2]:
df = explore.make_initial_df('repo_source')
df = explore.add_new_columns(df)

In [3]:
df.language.value_counts()

Python              7
JavaScript          6
Jupyter Notebook    5
TypeScript          2
Ruby                1
Swift               1
Vue                 1
PHP                 1
R                   1
HTML                1
Name: language, dtype: int64

In [4]:
vectorized_df = explore.make_vectorized_df(df)

In [5]:
X = vectorized_df
y = df.gen_language
#Slpit dataframe into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=.2, random_state = 123)

In [21]:
def logistic_regression(train, test):
    train = pd.DataFrame(dict(actual=y_train))
    test = pd.DataFrame(dict(actual=y_test))

    lm = LogisticRegression().fit(X_train, y_train)

    train['predicted'] = lm.predict(X_train)
    test['predicted'] = lm.predict(X_test)
    
    print('Accuracy: {:.2%}'.format(accuracy_score(train.actual, train.predicted)))
    print('---')
    print('Confusion Matrix')
    print(pd.crosstab(train.predicted, train.actual))
    print('---')
    print(classification_report(train.actual, train.predicted))

In [7]:
# make train and test precitions dataframe starting with actual values
# train_predictions = pd.DataFrame(y_train)
# train_predictions.columns = ['actual']
# test_predictions = pd.DataFrame(y_test)
# test_predictions.columns = ['actual']

# add baseline to predictions dataframe
# train_predictions['baseline'] = 'Python'
# test_predictions['baseline'] = 'Python'
# train_predictions

In [8]:
# test_predictions

In [9]:
# train, test = modeling.logistic_regression(df, vectorized_df)

TypeError: logistic_regression() missing 2 required positional arguments: 'y_train' and 'y_test'

In [ ]:
# train

In [10]:
rf = RandomForestClassifier(bootstrap=True, 
                            class_weight=None, 
                            criterion='gini',
                            min_samples_leaf=3,
                            n_estimators=100,
                            max_depth=3, 
                            random_state=123)

In [11]:
rf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=3, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=3, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=123,
                       verbose=0, warm_start=False)

In [12]:
train_random_forest_predictions = rf.predict(X_train)
test_random_forest_predictions = rf.predict(X_test)

In [16]:
knn = KNeighborsClassifier(n_neighbors=5, weights='uniform')

In [17]:
knn.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [18]:
train_knn_predictions = knn.predict(X_train)
test_knn_predictions = knn.predict(X_test)

NameError: name 'train' is not defined